<div style="background-color: #378CE7; border-radius: 15px; padding: 10px; margin: 10p; text-align: center;">
    <span style="font-size: 40px; color: white;">Lyrics Generator</span>
</div>

## IMPORTING LIBRARIES

In [16]:
#importing the libraries
import os
import gdown
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ydata_profiling # Provide a one-line Exploratory Data Analysis (EDA)
import nltk
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adamax
import warnings
warnings.filterwarnings("ignore")

## LOADING DATA

In [2]:
# URL to the dataset file on Google Drive
url = 'https://drive.google.com/uc?id=1O0MpSGku58I42hQYbN5NL7NrHE8N_Koi'

# Directory where you want to save the dataset
data_dir = 'data'

# Create the directory if it doesn't exist
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Path to save the dataset
file_path = os.path.join(data_dir, 'songs.csv')

# Download the dataset file
gdown.download(url, file_path, quiet=False)

print("Dataset downloaded and saved successfully.")

Downloading...
From: https://drive.google.com/uc?id=1O0MpSGku58I42hQYbN5NL7NrHE8N_Koi
To: /teamspace/studios/this_studio/RNN/data/songs.csv
100%|██████████| 1.07M/1.07M [00:00<00:00, 77.1MB/s]

Dataset downloaded and saved successfully.


In [3]:
# Importing the dataset
df=pd.read_csv("data/songs.csv")

# Checking the head of the data
df.head()

,Artist,Title,Lyrics
0,Taylor Swift,cardigan,"Vintage tee, brand new phone\nHigh heels on co..."
1,Taylor Swift,exile,"I can see you standing, honey\nWith his arms a..."
2,Taylor Swift,Lover,We could leave the Christmas lights up 'til Ja...
3,Taylor Swift,the 1,"I'm doing good, I'm on some new shit\nBeen say..."
4,Taylor Swift,Look What You Made Me Do,I don't like your little games\nDon't like you...


## DATA EXPLORATION

In [4]:
# Checking the info and if there is any null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745 entries, 0 to 744
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Artist  745 non-null    object
 1   Title   745 non-null    object
 2   Lyrics  745 non-null    object
dtypes: object(3)
memory usage: 17.6+ KB


In [5]:
# Generate a detailed report on the DataFrame 'df' using the pandas_profiling library
# This report includes statistics, summaries, and visualizations to understand the dataset
ydata_profiling.ProfileReport(df)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
# Retrieve the dimensions (number of rows and columns) of the DataFrame 'df'.
df.shape

(745, 3)

In [7]:
# Calculate the number of characters in each entry of the "Lyrics" column and
# store the result in a new column named "No_of_Characters"
df["No_of_Characters"] = df["Lyrics"].apply(len)

# Tokenize each entry of the "Lyrics" column into words using NLTK's word_tokenize function,
# then calculate the number of words in each tokenized list and store the result in a new column named "No_of_Words"
df["No_of_Words"] = df.apply(lambda row: nltk.word_tokenize(row["Lyrics"]), axis=1).apply(len)

# Split each entry of the "Lyrics" column by newline characters ('\n'),
# calculate the number of resulting lines, 
# and store the result in a new column named "No_of_Lines"
df["No_of_Lines"] = df["Lyrics"].str.split('\n').apply(len)

# Generate descriptive statistics for the dataset, 
# such as count, mean, standard deviation, minimum, 
# and maximum values for each numeric column, including the newly created columns
df.describe()

,No_of_Characters,No_of_Words,No_of_Lines
count,745.000000,745.000000,745.000000
mean,1403.347651,319.400000,46.277852
std,666.721467,156.163646,21.180531
min,1.000000,1.000000,1.000000
25%,946.000000,215.000000,33.000000
50%,1289.000000,291.000000,44.000000
75%,1714.000000,389.000000,56.000000
max,7267.000000,1652.000000,224.000000


## DATA PREPROCESSING

In [8]:
# Initialize an empty string to store the corpus
Corpus =''

# Iterate over each entry in the "Lyrics" column of the dataset and concatenate them into the Corpus string
for listitem in df.Lyrics:
    Corpus += listitem

# Convert the entire corpus to lowercase to standardize the text
Corpus = Corpus.lower()

# Calculate the number of unique characters in the corpus and print the result
print("Number of unique characters:", len(set(Corpus)))


Number of unique characters: 92


In [9]:
# Print the unique characters present in the corpus, sorted alphabetically
print("The unique characters:", sorted(set(Corpus)))

The unique characters: ['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '>', '?', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '~', '©', 'à', 'á', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'ñ', 'ó', 'ö', 'ü', 'ŏ', 'е', 'ا', 'س', 'ل', 'م', 'و', '\u2005', '\u200a', '\u200b', '–', '—', '‘', '’', '‚', '“', '”', '…', '\u205f', '\ufeff']


In [10]:
# List of symbols and characters to remove from the corpus
to_remove = ['{', '}', '~', '©', 'à', 'á', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'ñ', 'ó', 'ö', 'ü', 'ŏ',
             'е', 'ا', 'س', 'ل', 'م', 'و', '\u2005', '\u200a', '\u200b', '–', '—', '‘', '’', '‚', '“', '”', 
             '…', '\u205f', '\ufeff', '!', '&', '(', ')', '*', '-',  '/', ]

# Iterate over each symbol in the list to_remove and replace them with empty space in the Corpus
for symbol in to_remove:
    Corpus = Corpus.replace(symbol, " ")

# Print the unique characters present in the modified Corpus, after removing specified symbols
print("The unique characters:", sorted(set(Corpus)))

The unique characters: ['\n', ' ', '"', "'", ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '>', '?', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [11]:
# Sort the unique characters in the corpus and convert them to a list
symb = sorted(list(set(Corpus)))

# Get the length of the corpus and the length of the list of unique characters
L_corpus = len(Corpus) # Length of the corpus
L_symb = len(symb)     # Length of total unique characters

# Building a dictionary to map characters to indices and vice versa
mapping = dict((c, i) for i, c in enumerate(symb))          # Mapping from character to index
reverse_mapping = dict((i, c) for i, c in enumerate(symb))  # Mapping from index to character

# Print out the total number of characters and the number of unique characters
print("Total number of characters:", L_corpus)
print("Number of unique characters:", L_symb)

Total number of characters: 1045494
Number of unique characters: 47


In [12]:
length = 40  # Length of each sequence
features = []  # List to store the input sequences
targets = []   # List to store the corresponding targets

# Loop through the corpus to create sequences and targets
for i in range(0, L_corpus - length, 1):
    # Extract a sequence of characters from the corpus
    feature = Corpus[i:i + length]
    # Extract the next character in the corpus as the target for the current sequence
    target = Corpus[i + length]
    
    # Convert the sequence of characters to a list of corresponding indices using the mapping dictionary
    features.append([mapping[j] for j in feature])
    # Convert the target character to its corresponding index using the mapping dictionary
    targets.append(mapping[target])

# Calculate the total number of sequences created
L_datapoints = len(targets)

# Print the total number of sequences
print("Total number of sequences in the Corpus:", L_datapoints)

Total number of sequences in the Corpus: 1045454


In [13]:
# Reshape the input features to a 3D array with shape (L_datapoints, length, 1)
# and normalize the values by dividing by the total number of unique characters (L_symb)
X = (np.reshape(features, (L_datapoints, length, 1))) / float(L_symb)

# One-hot encode the output variable (targets)
y = to_categorical(targets)

## MODEL BUILDING

In [17]:
# Initialize the sequential model
model = Sequential()

# Adding layers to the model
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))  # LSTM layer with 256 units and input shape matching the shape of input features
model.add(Dense(y.shape[1], activation='softmax'))           # Dense layer with softmax activation function for output

# Compiling the model for training
opt = Adamax(learning_rate=0.01)  # Initialize the Adamax optimizer with a learning rate of 0.01
model.compile(loss='categorical_crossentropy', optimizer=opt)  # Compile the model using categorical cross-entropy loss

# Display a summary of the model architecture
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 256)            │       264,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 47)             │        12,079 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 276,271 (1.05 MB)

 Trainable params: 276,271 (1.05 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Train the model using the provided input features (X) and target labels (y)
# Specify the batch size and number of epochs
history = model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 36s 4ms/step - loss: 2.7157
Epoch 2/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 2.2864
Epoch 3/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 2.0805
Epoch 4/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 1.9674
Epoch 5/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 1.8847
Epoch 6/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 1.8251
Epoch 7/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 1.7654
Epoch 8/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 1.7395
Epoch 9/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 1.7029
Epoch 10/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 1.6945
Epoch 11/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 1.6715
Epoch 12/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 1.6250
Epoch 13/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 1.6366
Epoch 14/100
8168/8168 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - loss: 1.6091
E

In [25]:
# Save the training history to a file

import json
with open('training_history.json', 'w') as file:
    json.dump(history.history, file)

In [19]:
# Create a DataFrame from the history object containing loss values over epochs
history_df = pd.DataFrame(history.history)

# Plotting the learning curve of the model for loss
fig = plt.figure(figsize=(15, 4), facecolor="#B291B6")
fig.suptitle("Learning Plot of Model for Loss")  # Title of the plot
pl = sns.lineplot(data=history_df["loss"], color="#378CE7")  # Plotting the loss values
pl.set_ylabel("Training Loss")  # Label for the y-axis
pl.set_xlabel("Epochs")  # Label for the x-axis
plt.show()  # Show the plot

In [20]:
def Lyrics_Generator(starter, Ch_count):
    generated = ""  # Initialize an empty string to store the generated text
    starter = starter  # The starting sequence provided as input
    seed = [mapping[char] for char in starter]  # Convert the starter sequence into indices using the mapping dictionary
    generated += starter  # Append the starter sequence to the generated text

    # Generating new text of given length
    for i in range(Ch_count):
        seed = [mapping[char] for char in starter]  # Convert the current sequence into indices
        x_pred = np.reshape(seed, (1, len(seed), 1))  # Reshape the sequence for prediction
        x_pred = x_pred / float(L_symb)  # Normalize the sequence
        prediction = model.predict(x_pred, verbose=0)[0]  # Predict the next character probabilities

        # Applying temperature to the predictions
        prediction = np.asarray(prediction).astype('float64')
        prediction = np.log(prediction) / 1.0
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        
        # Sample from the predicted probabilities to get the index of the next character
        probas = np.random.multinomial(1, prediction, 1)
        index = np.argmax(prediction)
        next_char = reverse_mapping[index]  # Get the next character corresponding to the predicted index

        # Append the next character to the generated text
        generated += next_char
        starter = starter[1:] + next_char  # Update the starter sequence for the next iteration

    return generated  # Return the generated text

In [24]:
generated_song = Lyrics_Generator("learning from sprints", 400)

print(generated_song)

learning from sprints the rasers and the stars are the one thing that i dan do
i hot nn the str the way they say
i'm going to she was sav
it's a sane that i want to be a bab
but i know that i'm a brakd on the babk so men
of dance oi the sheets
of you and me
i can't stay the way they wanted the wild
and i want to be a bab
but i know that i'm a brakd on the babk so men
of dance oi the sheets
of you and me
i can't stay t


In [22]:
# Save the model to the native Keras format
model.save("lyrics_generator.keras")